In [1]:
import torch

In [12]:
import sys
import os
sys.path.append("..")
from globals import ROOT_DIR
from data_providers import TextDataProvider
import argparse
import configparser
from torch import optim
from experiment_builder import ExperimentBuilder
from data_providers import *
import os
from models.cnn import *
from models.multilayer_perceptron import multi_layer_perceptron
import tweepy
from utils import *
config = configparser.ConfigParser()
config.read('../config.ini')
path_data = os.path.join(ROOT_DIR, config['DEFAULT']['PATH_DATA'])
path_labels = os.path.join(ROOT_DIR, config['DEFAULT']['PATH_LABELS'])

consumer_key = config['DEFAULT']['TWITTER_CONSUMER_KEY']
consumer_secret_key = config['DEFAULT']['TWITTER_CONSUMER_SECRET_KEY']
access_token = config['DEFAULT']['TWITTER_ACCESS_TOKEN']
access_token_secret = config['DEFAULT']['TWITTER_ACCESS_TOKEN_SECRET']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [13]:
data = np.load(os.path.join(ROOT_DIR, 'data/founta_data.npy'))
data = data[()]
print(type(data))

<class 'dict'>


In [24]:
import time 
user_followers = {}  
count = 0 
for key, value in data.items():
    screen_name = value['user']['screen_name']
    ids = []
    for page in tweepy.Cursor(api.followers_ids, screen_name=screen_name).pages():
        ids.extend(page)
    user_followers[screen_name] = ids
    print("{} {}".format(screen_name, len(ids)))
    count += 1

fIoralhearts 2645
Joe_Flores02 275
Samiam3187 1300
steve45220 841


KeyboardInterrupt: 

2645
275
1300


In [ ]:
def extract_data(embedding_key, embedding_level, seed, experiment_flag):
    path_data = os.path.join(ROOT_DIR, config['DEFAULT']['PATH_DATA'])
    path_labels = os.path.join(ROOT_DIR, config['DEFAULT']['PATH_LABELS'])
    data_provider = TextDataProvider(path_data, path_labels, experiment_flag)

    if embedding_level == 'word':
        return data_provider.generate_word_level_embeddings(embedding_key, seed)
    elif embedding_level == 'char':
        output = data_provider.generate_char_level_embeddings(seed)
    elif embedding_level == 'tdidf':
        output = data_provider.generate_tdidf_embeddings(seed)

    return output

In [ ]:
data, data_map = extract_data('twitter', 'word', 28, 2)

In [ ]:
def wrap_data(batch_size, seed, data_local):
    train_set = DataProvider(inputs=data_local['x_train'], targets=data_local['y_train'], seed=seed)

    train_data_local = torch.utils.data.DataLoader(train_set,
                                                   batch_size=batch_size,
                                                   num_workers=2,
                                                   sampler=ImbalancedDatasetSampler(train_set))
    
    valid_set = DataProvider(inputs=data_local['x_valid'], targets=data_local['y_valid'], seed=seed)
    valid_data_local = torch.utils.data.DataLoader(valid_set,
                                                   batch_size=batch_size,
                                                   num_workers=2,
                                                   shuffle=False)

    test_set = DataProvider(inputs=data_local['x_test'], targets=data_local['y_test'], seed=seed)
    test_data_local = torch.utils.data.DataLoader(test_set,
                                                  batch_size=batch_size,
                                                  num_workers=2,
                                                  shuffle=False)

    return train_data_local, valid_data_local, test_data_local


In [ ]:
train_data, valid_data, test_data = wrap_data(2048, 28, data)

In [ ]:
import torch

out = torch.zeros([2048, 17, 400])
out.shape

In [ ]:
out = out.permute([0,2,1])
out = F.max_pool1d(out, out.shape[-1])
out = out.permute([0,2,1])
out.shape

In [ ]:
data_map['850023738779348994']

In [ ]:
list(data['x_train'].keys())[0]

In [ ]:
data['x_train']['847596956448960516']

In [ ]:
label_data = pd.read_csv(os.path.join(ROOT_DIR, 'data/founta_data.csv'), header='infer', index_col=0, squeeze=True).to_dict()

In [ ]:
len(label_data)

In [ ]:
outputs, labels = extract_tweets(label_data, data, 2)

In [ ]:
print(len(outputs))

In [ ]:
x_train, y_train, x_valid, y_valid, x_test, y_test = split_data(list(outputs.keys()), labels, 28)


In [ ]:
vectorizer = TfidfVectorizer(use_idf=True, max_features=10000)

In [ ]:
# embedded_tweets = vectorizer.fit_transform([outputs[key]['tweet'] for key in x_valid]).todense()
embedded_context_tweets = vectorizer.fit_transform([outputs[key]['context_tweet'] if outputs[key]['context_tweet'] is not None else '' for key in x_valid ]).todense()
for i, key in enumerate(x_valid):
    outputs[key]['embedded_tweet'] = embedded_tweets[i]
    outputs[key]['embedded_context_tweet'] = embedded_context_tweets[i]
print(outputs[key])

In [ ]:
for tweet in embedded_context_tweets:
    tweet = tweet + np.zeros(100).T
    print(len(tweet))
    break

In [ ]:
type(x_train_embed)

In [ ]:
x_train_embed = np.array(x_train_embed).reshape(x_train_embed.shape[0], 1, x_train_embed.shape[1])
x_train_embed.shape

In [ ]:
experiment_key = 2
data_provider = TextDataProvider(os.path.join(ROOT_DIR, 'data/founta_data.npy'), os.path.join(ROOT_DIR, 'data/founta_data.csv'), experiment_key)

In [ ]:
data = data_provider.generate_word_level_embeddings('bert', 28)

In [ ]:
x_train, y_train, x_valid, y_valid, x_test, y_test = split_data(outputs, labels, 28)

In [ ]:
data_embed = []
for j, output in enumerate(x_train):
    tweet_embed = data_provider.bert_embeddings[int(output['id'])]
    reply_status_id = int(output['in_reply_to_status_id'])
    blank_embed = []
    if reply_status_id == -1 or reply_status_id not in data_provider.bert_embeddings:
        for i in range(17):
            blank_embedding = np.zeros(768, )
            blank_embed.append(blank_embedding)
        data_embed.append(tweet_embed + blank_embed)
    else:
        data_embed.append(tweet_embed + data_provider.bert_embeddings[reply_status_id])
    if j > 10:
        break 

In [ ]:
np.array(data_embed).shape

In [ ]:
import torch

torch.Tensor(data_embed)

In [ ]:
embed = embeddings[int(x_train[0]['id'])]

In [ ]:
np.array(embed).reshape(1, embed[0], embed[1])

In [ ]:
happens = 0
data_embed = []
no_longer_available = 0
for output in outputs:
    tweet_embed = embeddings[int(output['id'])]
    reply_status_id = int(output['in_reply_to_status_id']) 
    final_embed = []
    if reply_status_id != -1 and reply_status_id not in embeddings:
        no_longer_available += 1
    if reply_status_id == -1 or reply_status_id not in embeddings:
        for i in range(17):
            blank_embedding = np.zeros(768, )
            final_embed.append(blank_embedding)
        final_embed = final_embed + tweet_embed
    else: 
        final_embed = final_embed + embeddings[reply_status_id]
    data_embed.append(final_embed)
    return data_embed

In [ ]:
print(data_embed[0].shape)

In [ ]:
status_ids = [value['in_reply_to_status_id'] for key, value in data.items() if value['in_reply_to_status_id'] is not None]
(len(status_ids) - len(replies)) / len(status_ids)

In [ ]:
#collected tweets from tweepy 
start_ptr = 0
end_ptr = start_ptr + 100 
replies = {}
status_ids = [output['in_reply_to_status_id'] for output in outputs]
print(len(status_ids))
count = 0
save_count = 0
while(start_ptr < len(status_ids)):
    print("Start ptr is at {}".format(start_ptr))
    reply_tweets = api.statuses_lookup(status_ids[start_ptr:end_ptr],trim_user=True)
#     if count % 1000 == 0:
#         print("Saving at {}".format(start_ptr))
#         np.savez(os.path.join(ROOT_DIR, 'data/reply_tweets{}.npz'.format(save_count)), a=replies)
#         save_count += 1
#         replies = {}
#     # load and save results each time 
#     if start_ptr != 0:
#         replies = np.load(os.path.join(ROOT_DIR, 'data/reply_data.npy'))
#         replies = replies[()]
    for i, reply_tweet in enumerate(reply_tweets):
        reply_tweet = reply_tweet._json
        replies[reply_tweet['id']] = reply_tweet['text']
        count += 1
        
    start_ptr += 100 
    end_ptr += 100

In [ ]:
len(replies)

In [ ]:
np.savez(os.path.join(ROOT_DIR, 'data/reply_data.npz'), a=replies)
replies = np.load(os.path.join(ROOT_DIR, 'data/reply_data.npy'))
replies = replies[()]
len(replies)

In [ ]:

count = 0 
# 0.16608010725657785 deleted... 
for status_id in status_ids:
    if str(status_id) not in replies:
        count += 1
print(count / len(status_ids))

In [ ]:
#verifies missing tweet 
# missing_tweet = api.statuses_lookup([847652506372984835],trim_user=True)
# missing_tweet
hey = '         '
hey.translate(str.maketrans('', '', string.punctuation))

In [ ]:
#combining tweets 
status_ids_fetched = []
outputs_context = []
for output in outputs:
    status_id = str(output['in_reply_to_status_id'])
    if status_id in replies:
        output['reply_to_tweet_text'] = output['text'] + replies[status_id]
    else:
        output['reply_to_tweet_text'] = output['text'] + output['text']
    outputs_context.append(output)

In [ ]:
import string 

def tokenize(outpus):
    key = 'reply_to_tweet_text'
    outputs_processed = []
    for output in outputs:
        text = output[key]
        text = text.translate(str.maketrans('', '', string.punctuation))
        tokens = text.split(' ')
        output['tokens'] = tokens
        outputs_processed.append(output)
    return outputs_processed

In [ ]:
outputs_processed = tokenize(outputs_context)
outputs_processed[0]

In [ ]:
GOOGLE_EMBED_DIM = 300
TWITTER_EMBED_DIM = 400
TWEET_SENTENCE_SIZE = 17*2 # 16 is average tweet token length
TWEET_WORD_SIZE = 20 # selected by histogram of tweet counts
FASTTEXT_EMBED_DIM = 300
EMBED_DIM = 200
NUM_CLASSES = 4

def generate_random_embedding(embed_dim):
    return np.random.normal(scale=0.6, size=(embed_dim,))

def process_tweet(tweet, embed_dim, word_vectors):
    embedded_tweet = []

    # trim if too large
    if len(tweet) >= TWEET_SENTENCE_SIZE:
        tweet = tweet[:TWEET_SENTENCE_SIZE]

    # convert all into word embeddings
    for word in tweet:
        embedding = generate_random_embedding(embed_dim) if word not in word_vectors else word_vectors[word]
        embedded_tweet.append(embedding)

    # pad if too short
    if len(tweet) < TWEET_SENTENCE_SIZE:
        diff = TWEET_SENTENCE_SIZE - len(tweet)
        embedded_tweet += [generate_random_embedding(embed_dim) for _ in range(diff)]
    return embedded_tweet

# embeds tokens! 
def fetch_word_embeddings(outputs, word_vectors, embed_dim, experiment_flag):
    outputs_embed = []
    for i, output in enumerate(outputs):

        # process first tweet
        embedded_tweet = process_tweet(output['tokens'], embed_dim, word_vectors)

        if experiment_flag == 2:
            #proceses second tweet
            if output['context_tweet'] is None:
                for i in range(TWEET_SENTENCE_SIZE):
                    blank_embedding = np.zeros(embed_dim,)
                    embedded_tweet.append(blank_embedding)
            else:
                context_embedding = process_tweet(output['context_tokens'], embed_dim, word_vectors)
                for i in range(TWEET_SENTENCE_SIZE):
                    embedded_tweet.append(context_embedding[i])
            assert len(embedded_tweet) == TWEET_SENTENCE_SIZE*2

        # add real numbers 
        if experiment_flag == 3:
            for i, embed in enumerate(embedded_tweet):
                print(np.array(embed).shape)
                embedded_tweet[i] = np.concatenate((embed, [output['retweet_count'], output['favorite_count']]))
                
        output['embedding'] = embedded_tweet
        print(np.array(embedded_tweet).shape)
        break
        outputs_embed.append(output)
    return outputs_embed

In [ ]:
start = time.time()
embed_dim = 400
filename = os.path.join(ROOT_DIR, 'data/word2vec_twitter_model/word2vec_twitter_model.bin')
word_vectors = KeyedVectors.load_word2vec_format(filename, binary=True, unicode_errors='ignore')
print("Total time {} min".format((time.time() - start) / 60))

In [ ]:
import time
start = time.time()
x_train_embed = fetch_word_embeddings(x_train, word_vectors, embed_dim, 3)
# x_valid_embed = fetch_word_embeddings(x_valid, word_vectors, embed_dim)
# x_test_embed = fetch_word_embeddings(x_test, word_vectors, embed_dim)
# print("Total time {} min".format((time.time() - start) / 60))
# print(len(x_train_embed), len(x_valid_embed), len(x_test_embed))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
def convert_to_feature_embeddings(x_embed, experiment_flag, key='embedding'):
    """
    :param x_embed: dictionary with all params of processed tweets
    :param key: what params should be kept
    :param experiment_flag: which type of experiment
    :return:
    """
    print("Experiment flag {}".format(experiment_flag))
    if key == 'tokens': #  for tdidf
        if experiment_flag == 1:
            return [x['tweet'] for x in x_embed]
        else:
            output = []
            for x in x_embed:
                if x['context_tweet']:
                    output.append(x['tweet'] + '\n' + x['context_tweet'])
                else:
                    output.append(x['tweet'] + '\n' + ' '.join([' '] * len(x['tweet'])))
            return output
    return [x[key] for x in x_embed]

vectorizer = TfidfVectorizer(use_idf=True, max_features=10000)
x_train_embed = vectorizer.fit_transform(convert_to_feature_embeddings(x_train,
                                                                       key='tokens',
                                                                       experiment_flag=3
                                                                       )).todense()
x_valid_embed = vectorizer.transform(convert_to_feature_embeddings(x_valid,
                                                                   key='tokens',
                                                                   experiment_flag=3
                                                                   )).todense()
x_test_embed = vectorizer.transform(convert_to_feature_embeddings(x_test,
                                                                  key='tokens',
                                                                          experiment_flag=3
                                                                          )).todense()

In [ ]:
def generate_tdidf_experiment_3_embeddings(data_list, embeddings):
    processed_embeddings = []
    for i, embed in enumerate(embeddings):
        embed = np.transpose(np.array(embed))
        embed = embed.reshape(-1)
        output = data_list[i]
        processed_embeddings.append(np.concatenate((embed, [output['retweet_count'], output['favorite_count']])))
    return processed_embeddings

x_train_embed = generate_tdidf_experiment_3_embeddings(x_train, x_train_embed)
print(np.array(x_train_embed).shape)

In [ ]:
x_train_embed[0].keys() # list of dictionaries 

In [ ]:
y_train[0] #int 

In [ ]:
# 17 n-gram 
# retweet count 
# in reply to status id 
# favorite count 

# 34 n-gram 
def convert_to_feature_embeddings(x_embed):
    return [x['word_embeddings'] for x in x_embed]
 
data = {}
print(x_train_embed[0].keys())
data['x_train'] = convert_to_feature_embeddings(x_train_embed)
data['y_train'] = y_train
data['x_valid'] = convert_to_feature_embeddings(x_valid_embed)
data['y_valid'] = y_valid
data['x_test'] = convert_to_feature_embeddings(x_test_embed)
data['y_test'] = y_test


In [ ]:
from torchvision import transforms

def wrap_data(batch_size, seed, x_train, y_train, x_valid, y_valid, x_test, y_test):
    transform = transforms.Compose([
        transforms.ToTensor(),
    ])
    transform=None
    
    train_set = DataProvider(inputs=x_train, targets=y_train, seed=seed, transform=transform)
    train_data_local = torch.utils.data.DataLoader(train_set,
                                                   batch_size=batch_size,
                                                   num_workers=2,
                                                   sampler=ImbalancedDatasetSampler(train_set),
                                                   )

    valid_set = DataProvider(inputs=x_valid, targets=y_valid, seed=seed, transform=transform)
    valid_data_local = torch.utils.data.DataLoader(valid_set,
                                                   batch_size=batch_size,
                                                   num_workers=2,
                                                   shuffle=False,
                                                  )

    test_set = DataProvider(inputs=x_test, targets=y_test, seed=seed, transform=transform)
    test_data_local = torch.utils.data.DataLoader(test_set,
                                                  batch_size=batch_size,
                                                  num_workers=2,
                                                  shuffle=False,
                                                 )
    return train_data_local, valid_data_local, test_data_local

def fetch_model(model, embedding_level, input_shape_local, dropout):
    if model == 'MLP':
        return multi_layer_perceptron(input_shape_local)
    if model == 'CNN':
        if embedding_level == 'word':
            return word_cnn(input_shape_local, dropout)
        elif embedding_level == 'character':
            return character_cnn(input_shape_local)
    if model == 'DENSENET':
        return densenet()
    else:
        raise ValueError("Model key not found {}".format(embedding_level))


def fetch_model_parameters(input_shape_local):
    model_local = fetch_model(model='CNN',
                            embedding_level='word',
                            input_shape_local=input_shape_local,
                            dropout=0.5)
    criterion_local = torch.nn.CrossEntropyLoss()
    optimizer_local = torch.optim.Adam(model_local.parameters(), weight_decay=1e-4)
    scheduler_local = optim.lr_scheduler.CosineAnnealingLR(optimizer_local, T_max=100, eta_min=0.0001)
    return model_local, criterion_local, optimizer_local, scheduler_local



In [ ]:
train_data, valid_data, test_data = wrap_data(2048, 28, **data)

In [ ]:
for x,y in train_data:
    input_shape = x.shape
    break 

In [ ]:
input_shape = tuple(input_shape)
input_shape

In [ ]:
def run_train_iter(model, device, optimizer, criterion, x, y, stats, experiment_key='train'):
    """
    Receives the inputs and targets for the model and runs a training iteration. Returns loss and accuracy metrics.
    :param x: The inputs to the model. A numpy array of shape batch_size, channels, height, width
    :param y: The targets for the model. A numpy array of shape batch_size, num_classes
    :return: the loss and accuracy for this batch
    """
    # sets model to training mode
    # (in case batch normalization or other methods have different procedures for training and evaluation)
    model.train()
    x = x.float()
    x = x.to(device)
    y = y.to(device)
    optimizer.zero_grad()  # set all weight grads from previous training iters to 0
    out = model.forward(x)  # forward the data in the model
    # loss = F.cross_entropy(input=out, target=y)  # compute loss
    loss = criterion(out, y)
    loss.backward()  # backpropagate to compute gradients for current iter loss

    optimizer.step()  # update network parameters
    _, predicted = torch.max(out.data, 1)  # get argmax of predictions
    accuracy = np.mean(list(predicted.eq(y.data).cpu()))  # compute accuracy
    stats['{}_acc'.format(experiment_key)].append(accuracy)
    stats['{}_loss'.format(experiment_key)].append(loss.data.detach().cpu().numpy())

def run_evaluation_iter(model, device, optimizer, criterion, x, y, stats, experiment_key='valid'):
    """
    Receives the inputs and targets for the model and runs an evaluation iterations. Returns loss and accuracy metrics.
    :param x: The inputs to the model. A numpy array of shape batch_size, channels, height, width
    :param y: The targets for the model. A numpy array of shape batch_size, num_classes
    :return: the loss and accuracy for this batch
    """
    with torch.no_grad():
        model.eval()  # sets the system to validation mode
        x = x.float()
        x = x.to(device)
        y = y.to(device)
        out = model.forward(x)  # forward the data in the model
        loss = criterion(out, y)
        
        # loss = F.cross_entropy(out, y)  # compute loss
        _, predicted = torch.max(out.data, 1)  # get argmax of predictions
        
        accuracy = np.mean(list(predicted.eq(y.data).cpu()))
        stats['{}_acc'.format(experiment_key)].append(accuracy)  # compute accuracy
        stats['{}_loss'.format(experiment_key)].append(loss.data.detach().cpu().numpy())
        

In [ ]:
def save_model(model, model_save_dir, model_save_name, model_idx):
    """
    Save the network parameter state and current best val epoch idx and best val accuracy.
    :param model_save_name: Name to use to save model without the epoch index
    :param model_idx: The index to save the model with.
    :param best_validation_model_idx: The index of the best validation model to be stored for future use.
    :param best_validation_model_acc: The best validation accuracy to be stored for use at test time.
    :param model_save_dir: The directory to store the state at.
    :param state: The dictionary containing the system state.

    """
    # Save state each epoch
    path = os.path.join(model_save_dir, "{}_{}".format(model_save_name, str(model_idx)))
    torch.save(model.state_dict(), f=path)
    

def load_model(model, model_save_dir, model_save_name, model_idx):
    """
    Load the network parameter state and the best val model idx and best val acc to be compared with the future val accuracies, in order to choose the best val model
    :param model_save_dir: The directory to store the state at.
    :param model_save_name: Name to use to save model without the epoch index
    :param model_idx: The index to save the model with.
    """
    path = os.path.join(model_save_dir, "{}_{}".format(model_save_name, str(model_idx)))
    checkpoint = torch.load(f=path)
    # freeze parameters
    model.load_state_dict(checkpoint)
    for parameter in model.parameters():
        parameter.requires_grad = False
    return model 

In [ ]:
from collections import OrderedDict, defaultdict
import tqdm

model, criterion, optimizer, _ = fetch_model_parameters(input_shape)
device = torch.device('cpu')
train_stats = OrderedDict()
num_epochs = 2

for epoch_idx in range(num_epochs):
    epoch_start_time = time.time()
    epoch_stats = defaultdict(list)
    with tqdm.tqdm(total=len(train_data)) as pbar_train:  # create a progress bar for training
        for idx, (x, y) in enumerate(train_data):  # get data batches
            run_train_iter(model, device, optimizer, criterion, x=x, y=y, stats=epoch_stats)  # take a training iter step
            pbar_train.update(1)
            pbar_train.set_description("loss: {:.4f}, accuracy: {:.4f}".format(epoch_stats['train_loss'][-1],
                                                                               epoch_stats['train_acc'][-1]))

    with tqdm.tqdm(total=len(valid_data)) as pbar_val:  # create a progress bar for validation
        for x, y in valid_data:  # get data batches
            run_evaluation_iter(model, device, optimizer, criterion, x=x, y=y, stats=epoch_stats)  # run a validation iter
            pbar_val.update(1)  # add 1 step to the progress bar
            pbar_val.set_description("loss: {:.4f}, accuracy: {:.4f}".format(epoch_stats['valid_loss'][-1],
                                                                             epoch_stats['valid_acc'][-1]))
     
    
    
    save_model(model, '', 'testing', epoch_idx)

In [ ]:
loaded_model = load_model(model, '', 'testing', 1)

#evaluate test here
with tqdm.tqdm(total=len(test_data)) as pbar_test:  # create a progress bar for validation
    for x, y in test_data:  # get data batches
        run_evaluation_iter(model, device, optimizer, criterion, x=x, y=y, stats=epoch_stats, experiment_key="test_experiment")  # run a validation iter
        pbar_test.update(1)  # add 1 step to the progress bar
        pbar_test.set_description("loss: {:.4f}, accuracy: {:.4f}".format(epoch_stats['test_experiment_loss'][-1],
                                                                  epoch_stats['test_experiment_acc'][-1]))


In [ ]:
epoch_stats

In [ ]:
arr = []
for i in range(17):
    arr.append(np.zeros(200,))

In [ ]:
np.array(arr).shape

In [ ]:
embedded_tweet = np.array(arr)

In [ ]:
embedded_tweet += np.array(arr)

In [ ]:
embedded_tweet.shape

In [ ]:
embedded_tweet = np.array(arr) + np.array(arr)

In [ ]:
embedded_tweet = []

In [ ]:
for i in range(17):
    blank_embedding = np.zeros(200,)
    embedded_tweet.append(blank_embedding)

In [ ]:
np.array(embedded_tweet).shape

In [ ]:
np.log(0)

In [ ]:
_shape = (2048, 24) 



In [ ]:
import numpy as np
x = np.zeros(_shape)
y = np.zeros(_shape)

In [ ]:
np.concatenate((x, y))

In [ ]:
import torch

x = torch.Tensor(x)
y = torch.Tensor(y)

In [ ]:
sol = torch.cat((x, y), 1)
sol

In [ ]:
sol.shape

In [ ]:
x.shape